# Create PDF
## How to use:
Run correctionplot.ipynb first. This will create all output needed for pdfs. Then simply run all cells.

Imports

In [ ]:
# NEED TO CLEAN IMPORTS!

from reportlab.platypus import Table, SimpleDocTemplate, TableStyle, Paragraph, Image, PageBreak, Frame
from reportlab.lib import colors
from reportlab.lib import utils
from reportlab.lib.units import cm

from os import listdir
from os.path import isdir

import csv

from reportlab.lib.pagesizes import landscape, A4
from reportlab.platypus.doctemplate import NextPageTemplate, PageTemplate, BaseDocTemplate

Definitions

In [ ]:
# makeTable takes data and creates a table with border and grid
def makeTable(data):
    table = Table(data)
    table.setStyle(TableStyle([('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),('BOX', (0,0), (-1,-1), 0.75, colors.black)]))
    return table

In [ ]:
# https://stackoverflow.com/questions/5327670/image-aspect-ratio-using-reportlab-in-python
def get_image(path, width=1*cm):
    img = utils.ImageReader(path)
    iw, ih = img.getSize()
    aspect = ih / float(iw)
    return Image(path, width=width, height=(width * aspect))

# NOT USED
def get_rimage(path, width=1*cm):
    img = utils.ImageReader(path)
    iw, ih = img.getSize()
    aspect = ih / float(iw)
    return RotatedImage(path, width=width, height=(width * aspect))

In [ ]:
# Pagenumber:
# https://code.activestate.com/recipes/546511-page-x-of-y-with-reportlab/
# https://code.activestate.com/recipes/576832/

# Header:
# https://stackoverflow.com/questions/49344094/reportlab-pass-custom-argument-to-canvas#49350581
# (specifically Barranka's answer)

from reportlab.pdfgen import canvas
from reportlab.lib.units import mm

class NumberedCanvas(canvas.Canvas):
    def __init__(self, *args, **kwargs):
        canvas.Canvas.__init__(self, *args, **kwargs)
        self._saved_page_states = []
        self.header = ID_global

    def showPage(self):
        self._saved_page_states.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        """add page info to each page (page x of y)"""
        num_pages = len(self._saved_page_states)
        for state in self._saved_page_states:
            self.__dict__.update(state)
            self.draw_page_number(num_pages)
            self.draw_header()
            canvas.Canvas.showPage(self)
        canvas.Canvas.save(self)

    def draw_page_number(self, page_count):
        self.setFont("Helvetica", 7)
        if self._pageNumber == 1:
            self.drawRightString(280*mm, 15*mm,
                "Page %d of %d" % (self._pageNumber, page_count))            
        else:
            self.drawRightString(200*mm, 15*mm,
                "Page %d of %d" % (self._pageNumber, page_count))

    def draw_header(self):
        self.setFont("Helvetica", 12)
        if self._pageNumber == 1:
            self.drawString(140*mm, 200*mm, self.header)
        else:
            self.drawString(100*mm, 280*mm, self.header)

# NOT USED: Can't get it to work, using templates instead!
# Rotate:
# https://stackoverflow.com/questions/29848988/a-simple-method-for-rotate-images-in-reportlab

# Center:
# https://stackoverflow.com/questions/43644835/reportlab-how-to-center-an-image-on-canvas
# (note, their solution used canvas)
class RotatedImage(Image):

    def wrap(self,availWidth,availHeight):
        h, w = Image.wrap(self,availHeight,availWidth)
        return w, h
    
    def draw(self):
        self.canv.rotate(90)
        Image.draw(self)

In [ ]:
# NOT USED
#https://stackoverflow.com/questions/14491169/reportlab-variable-nextpagetemplates

class fltpDocTemplate(BaseDocTemplate):
    def __init__(self, *args, **kwargs):
        BaseDocTemplate.__init__(self, *args, **kwargs)
    
    def afterPage(self):
        self._handle_nextPageTemplate('portrait')

In [ ]:
# NOT USED

def make_portrait(canvas,doc):
    canvas.setPageSize(A4)

def make_landscape(canvas,doc):
    canvas.setPageSize(landscape(A4))

In [ ]:
# Error handling

def error_switch(file):
    switch = {
        "errorname1": function1(file),
        "errorname2": function2(file),
    }
    func = switch.get(file, lambda: "No error-scenario matches file")
    return str(func)

def function1(file):
    #read info rom file, return str
    return "Incorrect number of laps detected: Expected 4, recieved "+str(10)+"<br/>"

def function2(file):
    #read info rom file, return str
    return "Incorrect number of laps detected: Expected 4, recieved "+str(10)+"<br/>"

In [ ]:
datafolder = '../data/'

# Used to scale images
png_width_full = 29
png_width_lap = 19

# Needed for header (was unable to send the information to the canvasmaker, so a global variable was the easiest solution)
global ID_global

for ID in listdir(datafolder):
    ID_global = ID
    output_location_string = datafolder+ID+'/'

    # Used to change template in the middle of document
    # https://stackoverflow.com/questions/50660395/reportlab-how-to-change-page-orientation/50660701
    # https://stackoverflow.com/questions/5913682/reportlab-how-to-switch-between-portrait-and-landscape
    # note: I set the borders by hand. The arguments are leftmargin, bottommargin, width, height
    portrait_frame = Frame(0, 0, 200*mm, 280*mm, id='portrait_frame')
    landscape_frame = Frame(0, 0, 297*mm, 200*mm, id='landscape_frame')

    # Because we are starting with landscape, we have to add the template at creation
    doc = BaseDocTemplate('report{}.pdf'.format(ID), pageTemplates=[PageTemplate(id='landscape', frames=landscape_frame, pagesize=landscape(A4)),PageTemplate(id='portrait', frames=portrait_frame, pagesize=A4)], pagesize=landscape(A4))
    
    elements = []
        
    #Add full race graph
    elements.append(get_image(output_location_string+'fullracepd{}.png'.format(ID), width=png_width_full*cm))

    #Add table
    with open(output_location_string+'correlation_{}.csv'.format(ID)) as c:
        data = list(csv.reader(c))
        elements.append(makeTable(data))
    
    #Error messages, if any
    if os.path.isdir(output_location_string+'errors/'):
        print("Error(s) found for: "+str(ID))
        errortext = "<br/>Note: " # Linebreak to increase distance from table
        for errorfile in listdir(output_location_string+'errors/'):
            errortext += error_switch(errorfile)
        errorpar = Paragraph(errortext) #might need to specify style
        elements.append(errorpar)
        
    #Switch template (change happens on next page, hence before pagebreak)
    elements.append(NextPageTemplate('portrait'))
    elements.append(PageBreak())

    #Find all lap plots
    keyword = 'lap'
    lap_plot_list = []
    for lap_plot_file in listdir(output_location_string):
        if keyword in lap_plot_file:
            lap_plot_list.append(lap_plot_file)

    #Sort and add graphs
    lap_plot_list.sort()
    for lap_graph in lap_plot_list:
        elements.append(get_image(output_location_string+lap_graph, width=png_width_lap*cm))

    doc.build(elements, canvasmaker=NumberedCanvas)